In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cv2 as cv
from tqdm import tqdm
import os
import json

from lac.plotting import plot_path_3d, plot_3d_points
from lac.localization.mono_vo import MonoVisualOdometry

%load_ext autoreload
%autoreload 2

# Feature detection


In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/data_collection_1")

i = 100
I1 = cv.imread(os.path.join(data_path, "front_left", f"{i}.png"), cv.IMREAD_GRAYSCALE)
I2 = cv.imread(os.path.join(data_path, "front_left", f"{i + 10}.png"), cv.IMREAD_GRAYSCALE)

# Plot images side by side
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(I1)
ax[1].imshow(I2)
plt.show()

ORB (left) vs FAST (right)


In [ ]:
fast_detector = cv.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
orb_detector = cv.ORB_create(nfeatures=1000)
orb_kp = orb_detector.detect(I1)
orb_kp, orb_des = orb_detector.compute(I1, orb_kp)

fast_kp = fast_detector.detect(I1)

I1_orb = cv.drawKeypoints(I1, orb_kp, None, color=(0, 255, 0), flags=0)
I1_fast = cv.drawKeypoints(I1, fast_kp, None, color=(0, 255, 0), flags=0)
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(I1_orb)
ax[1].imshow(I1_fast)
plt.show()

In [ ]:
I2_fast = cv.drawKeypoints(I2, fast_detector.detect(I2), None, color=(0, 255, 0), flags=0)
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(I1_fast)
ax[1].imshow(I2_fast)
plt.show()

# Mono VO class


In [ ]:
## Parameters ---------------------------------------------------------------

data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/data_collection_1")
img_path = f"{data_path}/front_left"
json_data = json.load(open(f"{data_path}/data_log.json"))
initial_pose = np.array(json_data["initial_pose"])  # TODO: initialize with initial pose

trajlen = 1000

# for KITTI
focal = 915.0
pp = (1280 / 2, 720 / 2)  # principal point

# Parameters for lucas kanade optical flow
lk_params = dict(
    winSize=(21, 21), criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 30, 0.01)
)

In [ ]:
poses = []
positions = []
for frame in json_data["frames"]:
    poses.append(np.array(frame["pose"]))
    positions.append(np.array(frame["pose"])[:3, 3])
positions = np.array(positions)

In [ ]:
vo = MonoVisualOdometry(img_path, poses, focal, pp, lk_params)
# vo.R = initial_pose[:3, :3]
# vo.t = initial_pose[:3, 3][:,None]
vo.initialize_pose(initial_pose[:3, :3], initial_pose[:3, 3][:, None])
vo_traj = np.zeros((trajlen, 3))

for i in tqdm(range(trajlen)):
    vo.process_frame()
    vo_traj[i, :] = vo.get_mono_coordinates()

In [ ]:
fig = plot_path_3d(vo_traj[:, 0], vo_traj[:, 1], vo_traj[:, 2])
fig = plot_path_3d(positions[:trajlen, 0], positions[:trajlen, 1], positions[:trajlen, 2], fig=fig)
fig.show()

# IMU + lander measurements


In [2]:
casename = "20hz_seed4"
data_path = "../../output/imu_20hz/" + "data_log.json"
json_data = json.load(open(f"{data_path}"))

poses = []
imu_data = []

for frame in json_data["frames"]:
    poses.append(np.array(frame["pose"]))
    imu_data.append(np.array(frame["imu"]))

imu_data = np.array(imu_data)

In [3]:
from lac.util import skew_symmetric, normalize_rotation_matrix

In [4]:
R_0 = poses[0][:3, :3]

imu_rotations = []
imu_rotations.append(R_0)

for i in range(1, len(poses)):
    w_hat = skew_symmetric(imu_data[i, 3:])
    dt = json_data["frames"][i]["mission_time"] - json_data["frames"][i - 1]["mission_time"]
    R_1 = (np.eye(3) - w_hat * dt).T @ R_0
    R_1 = normalize_rotation_matrix(R_1)
    imu_rotations.append(R_1)
    R_0 = R_1

In [5]:
delta_rotations = []
for i in range(1, len(imu_rotations)):
    delta_rotations.append(imu_rotations[i] @ imu_rotations[i - 1].T)

In [6]:
gravity = np.array([0.0, 0.0, 1.6220])  # m/s^2

use_true_rot = False

t_0 = poses[0][:3, 3]
t_1 = poses[1][:3, 3]
v_1 = (t_1 - t_0) / (
    json_data["frames"][1]["mission_time"] - json_data["frames"][0]["mission_time"]
)

imu_translations = []
imu_translations.append(t_0)
imu_translations.append(t_1)

for i in range(2, len(poses)):
    dt = json_data["frames"][i]["mission_time"] - json_data["frames"][i - 1]["mission_time"]
    a = imu_data[i, :3]

    if use_true_rot:
        R_i = poses[i][:3, :3]
    else:
        R_i = imu_rotations[i]

    v_2 = v_1 + (R_i @ a - gravity) * dt
    t_2 = t_1 + v_1 * dt

    imu_translations.append(t_2)
    t_0 = t_1
    t_1 = t_2
    v_1 = v_2

imu_translations = np.array(imu_translations)

In [7]:
delta_translations = []
for i in range(1, len(imu_translations)):
    delta_translations.append(imu_translations[i] - imu_translations[i - 1])

In [8]:
initial_pose = np.array(json_data["initial_pose"])
lander_pose_rover = np.array(json_data["lander_pose"])
lander_pose_world = initial_pose @ lander_pose_rover

In [9]:
lander_position = lander_pose_world[:3, 3]

# Lander range and bearing measurements
lander_pose_measurements = []  # relative pose of lander in rover frame
lander_range_measurements = []
lander_los_measurements = []

for i in range(len(poses)):
    t_i = poses[i][:3, 3]
    delta_t = t_i - lander_position
    lander_range_measurements.append(np.linalg.norm(delta_t))
    lander_pose_measurements.append(poses[i].T @ lander_pose_world)
    lander_los_measurements.append(-delta_t / np.linalg.norm(delta_t))

In [10]:
import symforce

try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass
import symforce.symbolic as sf

from lac.localization.symforce_util import odometry_lander_fgo

In [35]:
ODOM_R_SIGMA = 0.01  # for rotations
ODOM_T_SIGMA = 0.1  # for translations [m]
ODOM_SIGMA = np.ones(6)
ODOM_SIGMA[:3] *= ODOM_R_SIGMA
ODOM_SIGMA[3:] *= ODOM_T_SIGMA

init_poses = []
imu_odometry = []
for i in range(len(poses)):
    init_pose = np.eye(4)
    init_pose[:3, :3] = imu_rotations[i]
    init_pose[:3, 3] = imu_translations[i]
    init_poses.append(init_pose)

for i in range(len(poses) - 1):
    imu_odom = np.eye(4)
    imu_odom[:3, :3] = delta_rotations[i]
    imu_odom[:3, 3] = delta_translations[i]
    imu_odometry.append(imu_odom)

In [ ]:
len(poses)

In [ ]:
N = 2000

opt_poses, result = odometry_lander_fgo(
    init_poses[:N],
    lander_pose_world,
    imu_odometry[: N - 1],
    # lander_pose_measurements[:N],
    lander_los_measurements[:N],
    ODOM_SIGMA,
    1.0,
    debug=True,
)

In [ ]:
# N = -1

opt_positions = []
for pose in opt_poses:
    opt_positions.append(pose[:3, 3])
opt_positions = np.array(opt_positions)

init_positions = []
for pose in init_poses:
    init_positions.append(pose[:3, 3])
init_positions = np.array(init_positions)

gt_translations = np.zeros((len(poses), 3))
for i in range(len(poses)):
    gt_translations[i] = poses[i][:3, 3]

fig = plot_path_3d(
    opt_positions[:, 0], opt_positions[:, 1], opt_positions[:, 2], color="orange", name="Optimized"
)
fig = plot_path_3d(
    gt_translations[:N, 0],
    gt_translations[:N, 1],
    gt_translations[:N, 2],
    fig=fig,
    color="blue",
    name="Ground truth",
)
fig = plot_path_3d(
    init_positions[:N, 0],
    init_positions[:N, 1],
    init_positions[:N, 2],
    fig=fig,
    color="green",
    name="IMU initialization",
)
fig = plot_3d_points(lander_position[None, :], fig=fig, color="red", markersize=3)
fig.update_layout(height=700, width=1200, scene_aspectmode="data")
fig.show()

In [ ]:
lander_position[None, :]